![LU Logo](https://www.df.lu.lv/fileadmin/user_upload/LU.LV/Apaksvietnes/Fakultates/www.df.lu.lv/Par_mums/Logo/DF_logo/01_DF_logo_LV.png)

# Intermediate Python

## Lesson Overview

We will cover the following topics:

* Decorators
* Context managers
* Generators, generator expressions

## Lesson Prerequisites

* Working knowledge of Python - what has been covered in this course so far.

## Lesson Objectives

At the end of this lesson you should be able to:

* Write and use decorators
* Write and use context managers
* Write and use generators and generator expressions


### Import required libraries

In [ ]:
# generally imports go at the top of a notebook
# python version
import sys
print(f"Python version: {sys.version}")

### Topic 1: - Decorators

**Python decorators** are a design pattern that allow us to extend or to modify the behavior of callable objects (like functions or methods) without permanently altering the callable itself. Essentially, decorators wrap or "decorate" a function, enabling pre- and post-processing actions around the original function call. 

Decorators can be used to extend existing functions and methods with functionality such as logging, caching or access control.

Python decorators are applied using the "@" syntax above the function or method definition:

```
@decorator_name
def my_function():
    ...
```


#### 1.1 – Higher-Order Functions

In Python, functions are "first-class" objects. They can be assigned to variables, passed as arguments and returned from functions (just like any other object).

Higher-order functions are functions that work with other functions: they may accept functions as their arguments, they may return functions, or both. In this example `func_2(my_function)` is a higher-order function.

In [7]:
def func_1(text):
    print("In func_1:", text)

def func_2(my_function):
    print("In func_2:")
    return my_function("Hi!")

func_1("Hello there!")
print()

func_2(func_1)

In func_1: Hello there!

In func_2:
In func_1: Hi!


Here we can see that `func_2()` executes whatever function is passed to it.

A function may also define new functions (called *inner* or *nested functions*) inside it as well as return a function:

In [8]:
def parent_fn():
    print("Inside the parent_fn() function.")

    def child_fn(text):
        print("Inside the child_fn() function:", text)

    # returning a function
    return child_fn

parent_fn()

Inside the parent_fn() function.


<function __main__.parent_fn.<locals>.child_fn(text)>

In [9]:
# parent_fn() returns a function
my_fn = parent_fn()
print()

# we can execute the function returned by parent_fn()
my_fn("Hello!")

Inside the parent_fn() function.

Inside the child_fn() function: Hello!


#### 1.2 – Function decorators

Function decorators "wrap" a function or a class method and modifies or extends its functionality.

Using *higher-order functions* we can define a function that takes another function as an argument and returns back another function:

In [10]:
def my_decorator(func):
    
    # function that calls func() provided as an argument to my_decorator()
    def wrapper():
        
        print("Do something before the function is called.")

        # call the original function
        func()
        
        print("Do something after the function is called.")

    # returns the wrapper funcion
    return wrapper

In [11]:
# define a new function
def my_fn():
    print("Executing my_fn() function.")

my_fn()

Executing my_fn() function.


Decorators wrap a function, modifying its behavior:

In [12]:
# now we can re-define the function by "wrapping" it in decorator's inner function
my_fn = my_decorator(my_fn)

my_fn()

Do something before the function is called.
Executing my_fn() function.
Do something after the function is called.


In [13]:
# Python decorators let us decorate functions by using "@" syntax

# @my_decorator means the same as my_fn2 = my_decorator(my_fn2)
@my_decorator
def my_fn2():
    print("In the original function.")

my_fn2()

Do something before the function is called.
In the original function.
Do something after the function is called.


You can also define decorators in your own modules, then import and use them:

In [14]:
%%writefile decorator_module.py

def do_twice(func):

    def wrapper():
        func()
        func()

    # returns the wrapper funcion
    return wrapper

Writing decorator_module.py


In [15]:
from decorator_module import do_twice

@do_twice
def my_fn3():
    print("Hello, world!")

my_fn3()

Hello, world!
Hello, world!


#### Functions with arguments and return values

If the decorated function accepts arguments or returns a value, the decorator must also handle these arguments and the return value.

The decorated function may have an arbitrary number of positional and keyword arguments. To get the values of these arguments, we can use the `*args` (for positional arguments) and `**kwargs` (for keyword arguments) syntax that handles a variable number of arguments:

In [17]:
def do_twice_args(func):

    # supply arguments to the inner function
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)

    # returns the wrapper funcion
    return wrapper

In [18]:
@do_twice_args
def print_name(name):
    print(f"Name: {name}")

print_name("John")

Name: John
Name: John


---

You can also decorate functions that return a value:

In [20]:
def log_runs(func):

    # supply arguments to the inner function
    def wrapper(*args, **kwargs):
        print("Before the function call.")
        result = func(*args, **kwargs)
        print("After the function call. Return value:", result)

        return result

    return wrapper

In [22]:
@log_runs
def multiply_10x(number, my_name="Nothing"):
    print(f"In multiply_10x function. Name: {my_name}")
    return number * 10

multiply_10x(150, my_name="Uldis")

Before the function call.
In multiply_10x function. Name: Uldis
After the function call. Return value: 1500


1500

---

Python functions normally know their name and attributes, and Python can display documentation (help) related to a function. However, when decorated, functions "loose" this information. Instead, information about the wrapper function is displayed:


In [23]:
multiply_10x

<function __main__.log_runs.<locals>.wrapper(*args, **kwargs)>

In [24]:
help(multiply_10x)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    # supply arguments to the inner function



Let's fix that using the `functools.wraps` decorator:

In [25]:
import functools

def log_runs2(func):

    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Before the function call.")
        result = func(*args, **kwargs)
        print("After the function call. Return value:", result)

        return result

    return wrapper

In [26]:
@log_runs2
def multiply_15x(number):
    """Multiply the number by 15."""
    
    print("In multiply_15x function.")
    return number * 15

multiply_15x(150)

Before the function call.
In multiply_15x function.
After the function call. Return value: 2250


2250

Python can now correctly display the name of the function and its help message:

In [27]:
multiply_15x

<function __main__.multiply_15x(number)>

In [28]:
help(multiply_15x)

Help on function multiply_15x in module __main__:

multiply_15x(number)
    Multiply the number by 15.



#### 1.3 – Decorator Examples

Python decorators can be used for various purposes including:
- **Logging** information about function execution 
- **Authorization** – checking if the user is authorized to use the function
- **Caching** – storing results of expensive function calls and returning the cached result when requested
- **Validation** – checking the inputs or outputs of a function

Multiple decorators can be applied to the same function if necessary.

##### Flask web framework

[Flask](https://flask.palletsprojects.com/en/2.3.x/), a web micro-framework for Python, uses decorators for route definitions and other purposes.

Here is a basic example of Flask returning "Hello, Flask!" when the website's root is accessed. It employs a `@app.route()` decorator to associate a function with the corresponding web URL it serves:

```
from flask import Flask

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello, Flask!"
```

We could also define a custom decorator to ensure that a user is logged in before they can access a specific route:

```
from flask import Flask, g, request, redirect, url_for
import functools

app = Flask(__name__)

def login_required(func):
    """Make sure user is logged in before proceeding"""
    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for("login", next=request.url))
        return func(*args, **kwargs)
    return wrapper_login_required
```

Then you can use this decorator to make sure a user is logged in:

```
@app.route('/secret')
@login_required
def secret():
    return "Welcome to this secret webpage!"
```

##### Dataclasses

The `dataclass()` decorator, introduced in Python 3.7, provides a way to declare [**data classes**](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass). Data classes are classes containing mainly data. A data class describes its attributes using class variable annotations.

```
from dataclasses import dataclass

@dataclass
class Point:
    x: float
    y: float
    z: float = 0.0

p = Point(1.5, 2.5)

# prints "Point(x=1.5, y=2.5, z=0.0)"
print(p) 
```

This is an example of applying a decorator to a class.

##### Caching

Decorators can be used for caching by storing the results of function calls in a cache.

When the function is called again with the same arguments, the decorator retrieves the result from the cache rather than executing the function, optimizing performance and reducing redundant computations. 

This technique, also called *memoization* is especially useful for expensive or recursive functions. An example of a recursive function that can benefit from caching is the recursive Fibonacci sequence function.

Python standard library includes a least-recently-used (LRU) cache as a [@functools.lru_cache](https://docs.python.org/library/functools.html#functools.lru_cache) decorator.

In [29]:
import functools

@functools.lru_cache(maxsize=10)
def fibonacci(num):
    
    print(f"Calculating fibonacci({num})")
    
    if num < 2:
        return num
        
    return fibonacci(num - 1) + fibonacci(num - 2)

In [30]:
# "Calculating fibonacci" is printed every time the function is executed
fibonacci(10)

Calculating fibonacci(10)
Calculating fibonacci(9)
Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


55

In [31]:
# not printing "Calculating fibonacci" this time as the return value has already been cached
fibonacci(8)

21

### Topic 2: - Context Managers

**Context managers** allow you to allocate and release resources precisely when you want to, ensuring that resource setup and teardown actions are reliably executed.

Context managers have a variety of uses including:
* **file operations** – properly closing open files
* **database connections** – ensuring that database connections are properly closed or committed / rolled back
* **thread safety** – acquiring and releasing locks to ensure that resources are accessed in a thread-safe manner

---

Context managers are commonly used with the `with` statement. The `with` statement is commonly used when working with files:

```
with open("test_file.txt", "r") as file:
    text = file.read()
```

Here the `with` statement ensures that the opened file is properly closed, even if an exception is raised. Closing the file takes place when the program exits the `with` command block: `text = file.read()`

You can close the file manually without a context manager, as demonstrated in the following code. However, if an exception occurs during the file.read() call, the file won't be automatically closed.

```
file = open("test_file.txt", "r")
text = file.read()
file.close()
```

In order to properly close the file, you would need to use a `try: ... finally:` statement:

```
file = open("test_file.txt", "r")

try:
    content = file.read()

finally:
    file.close()
```

Context managers simplify resource management in Python, leading to more readable and error-resistant code.

---

A **context manager** can be implemented:
* using a context manager protocol (interface)
* using a generator function

**Context manager protocol** lets you define a context manager by implementing two special methods: `__enter__` and `__exit__`. These methods allow developers to set up and tear down the resources managed by the context manager.

In [32]:
class FileOpener:
    
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None

    def __enter__(self):
        print(f"Opening: {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            print(f"Closing: {self.filename}")
            self.file.close()

In [33]:

with FileOpener("decorator_module.py", "r") as in_file:
    print("Reading the file.")
    content = in_file.read()

Opening: decorator_module.py
Reading the file.
Closing: decorator_module.py


---

Context managers can be used to **manage database transactions**:

```
class DatabaseTransaction:
    def __init__(self, connection):
        self.connection = connection

    def __enter__(self):
        return self.connection.cursor()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            self.connection.commit()
        else:
            self.connection.rollback()
```
Assuming that `conn` is an already established database connection:

```
with DatabaseTransaction(conn) as cursor:
    cursor.execute('INSERT INTO table (col1, col2) VALUES (?, ?)', (val1, val2))
```

---

**Timer context manager** lets you measure the time taken by a block of code to execute.

In [34]:
import time

class CodeTimer:
    def __enter__(self):
        self.start_time = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        print(f'Code took {elapsed_time:.2f} seconds to execute')


In [35]:
with CodeTimer():
    # some time-consuming operations
    result = [i**2 for i in range(1000000)]

Code took 0.35 seconds to execute


In [36]:
# Jupyter also has a "magic" keyword for measuring the duration of code execution.

%time result = [i**2 for i in range(1000000)]

print()

%timeit result = [i**2 for i in range(1000000)]

CPU times: user 311 ms, sys: 15 ms, total: 326 ms
Wall time: 329 ms

310 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Topic 3: - Generators

Generator functions allow us to declare a function that behaves like an iterator (e.g. it can be used in a for loop).

These functions can generate (yield) values for each iteration of the loop instead of returning a single value at the end of the function.

https://wiki.python.org/moin/Generators

* Generators (generator functions)
* Generator expressions

Generator functions are defined the same way as other Python functions
- except that they use `yield` command to return values for each iteration of the loop that uses this generator function

---

Generators:
- are simpler than regular functions written to do the same task
- use less memory space because return values are calculated on-the-fly without a need to save all the values to a list
- are "lazy" – they only generate values when asked to

Generators can produce data that is huge or infinite.

#### 3.1 – Example

In [37]:
# First n numbers - using a regular function

def first_n(n):
    '''Build and return a list'''
    num, nums = 0, []
    while num < n:
        nums.append(num)
        num += 1
    return nums

In [39]:
res = first_n(1000_000)

# how much memory does it use?
import sys
print(sys.getsizeof(res))
print()

# sum of all the numbers
print(sum(res))

8448728

499999500000


In [41]:
# First n numbers - using a generator function

def first_n_gen(n):
    num = 0
    while num < n:
        yield num
        num += 1

# Generator yields items (as they are requested) instead of returning a list 
# at the end of the function

In [42]:
res_gen = first_n_gen(1_000_000)

# the result is a generator
print(type(res_gen))

print()
print(sys.getsizeof(res_gen))

<class 'generator'>

112


In [43]:
# generator generates values one-by-one, on demand

# calculate the sum of all the values
print(sum(res_gen))

499999500000


In [44]:
# once all generator values have been requested,
# it is "used up" - no values remain to return

# so we can not iterate through a generator twice in a row

print(sum(res_gen))
print()

# but we can create a new generator object and iterate through it

print(sum(first_n_gen(1_000_000)))

0

499999500000


---

#### 3.2 – Generator implementation

Iterators are objects that allow you to "step over" (iterate) items in collections of data.

Generators implement the iterator protocol "under the hood":
- `__iter__()` is called to initialize the iterator. It must return an iterator object (typically it returns `self`)
- `__next__()` is used to iterate over the iterator (it returns the next iterator value)
- once the data stream has ended, an iterator must raise a `StopIteration` exception
 
Once a generator has been created, you can loop over it using a `for` loop or get the values directly using Python's `next()` function.

In [45]:
res = first_n_gen(10)

for i in res:
    print(i)

0
1
2
3
4
5
6
7
8
9


In [46]:
# we could implement an iterator directly
# but that would require more code than necessary

# generators make this code simpler and easier to understand

class first_n_iter:

    def __init__(self, n):
        self.n = n
        self.num = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.num < self.n:
            cur, self.num = self.num, self.num+1
            return cur
            
        raise StopIteration()

print(sum(first_n(1_000_000)))

499999500000


In [47]:
# methods and attributes that a generator object has:

my_data = first_n_gen(5)

print(dir(my_data))

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


In [48]:
# Generators are lazy and you can only iterate through them once

next_val = next(my_data)
print(next_val)

0


In [49]:
print(next(my_data))
print(next(my_data))
print(next(my_data))
print(next(my_data))

1
2
3
4


In [50]:
# Data has finished, this will raise an exception
print(next(my_data))

StopIteration: 

In [51]:
# for loops know how to use the iterator protocol
# (e.g. they handle StopIteration exception)

for item in first_n_gen(5):
    print(item)

0
1
2
3
4


In [ ]:
# let's add some print() calls

def my_gen_fn(n):
    print("In generator function")

    num = 0
    
    while num < n:
        print("- before yield")
        yield num
        print("- after yield")
        
        num += 1

    print("End of the function")

In [52]:
my_gen = my_gen_fn(6)

In [53]:
# let's get some values from this generator
# what message gets printed when?

print(next(my_gen))
print()


In generator function
- before yield
0



In [54]:
print(next(my_gen))
print()

- after yield
- before yield
1



---

**Generators can be infinite** (e.g. to generate infinite sequences):
- then the program using the generator has to limit the number of iterations and make sure its execution finishes at some point

In [56]:
# Fibonacci sequence – an infinite generator

def fibonacci_gen():
    """
    Generates a Fibonacci sequence.
    """
    a, b = 0, 1

    while True:
        yield a
        a, b = b, a+b

In [57]:
# we can use islice() to limit the length of the iterator

# https://docs.python.org/3/library/itertools.html#itertools.islice
from itertools import islice

my_numbers = fibonacci_gen()
my_numbers = islice(my_numbers, 20)

for item in my_numbers:
    print(item)

0
1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181


In [58]:
# you can use multiple yield statements
# ... but in this example you might as well use a list

def city_gen():
    yield "Rīga"
    yield "Liepāja"
    yield "Valmiera"

for item in city_gen():
    print(item)

Rīga
Liepāja
Valmiera


---

#### 3.3 – Generator expressions

Generator expressions are similar to list comprehension but with parenthesis "()" instead of square brackets "[]".

Generator expressions create a generator object instead of a list and they:
- use "lazy" iteration
- save memory by not constructing the whole list in memory

In [59]:
squares = [n**2 for n in range(50)]

print(sys.getsizeof(squares))
print()

print(squares)

472

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401]


In [60]:
squares_gen = (n**2 for n in range(50))

print(sys.getsizeof(squares_gen))
print(type(squares_gen))
print()

print(squares_gen)

112
<class 'generator'>

<generator object <genexpr> at 0x10c96e430>


In [61]:
print(list(squares_gen))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401]


---

#### 3.4 – Generators as Data Processing Pipelines

[**Generator Tricks for Systems Programmers, v3.0**](https://www.dabeaz.com/generators/)
- by David M. Beazley
- Looks at various techniques for using generator functions and generator expressions in the context of systems programming (processing log files, text parsing, etc.)

https://github.com/dabeaz/generators

https://www.dabeaz.com/generators/Generators.pdf

---

**Task**: Find out how many bytes of data were
transferred by summing up the last column
of data in the Apache web server log

```
81.107.39.38 - ... "GET /favicon.ico HTTP/1.1" 404 133
81.107.39.38 - ... "GET /ply/bookplug.gif HTTP/1.1" 200 23903
81.107.39.38 - ... "GET /ply/ply.html HTTP/1.1" 200 97238
```

---

**See David Beazley's presentation from slide #30**

```
with open("access-log") as wwwlog:
    bytecolumn = (line.rsplit(None,1)[1] for line in wwwlog)
    bytes_sent = (int(x) for x in bytecolumn if x != '-')
    print("Total", sum(bytes_sent))
```

**This data processing pipeline is implemented using generator expressions.**

```
open("access-log") => bytecolumn => bytes_sent => sum()
```

---

#### Topic 3 - mini exercise

Write a generator that works like a `range()` function but for floating point values.


## Lesson Overview

What have we learned?

* Decorators - what are they and how to use them
* Context managers - what are they and how to use them
* Generators, generator expressions - what are they and how to use them

## Exercises for further practice

### Exercise 1 - Write Your Own Decorator

Write a Decorator that prints your name before and after the decorated function is called.

### Exercise 2 - Write Your Own Context Manager

* Write a Context Manager that prints your name before and after the code block is executed.
* Add time measurement to the context manager.

### Exercise 3 - Use Generators to Process Data

Use generators to write a data processing pipeline that consists of the following steps:
- open a text file
- split text into words (by creating a generator that yields tokens / words)
- filter text for words that are more than 3 characters long
- count the number of occurrences of each word
- finally, print resulting statistics of word occurrences

## Additional Resources

### Topic 1 - resources

- [Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/)
- [@functools.wraps](https://docs.python.org/3/library/functools.html#functools.wraps) decorator
- [Flask Quickstart](https://flask.palletsprojects.com/en/2.3.x/quickstart/)
- [Data Classes in Python 3.7+](https://realpython.com/python-data-classes/)

### Topic 2 - resources

- [Context Manager](https://realpython.com/python-with-statement/)

### Topic 3 - resources

- [Python Wiki: Generators](https://wiki.python.org/moin/Generators)
- [How to Use Generators and yield in Python](https://realpython.com/introduction-to-python-generators/) – Real Python